In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np

In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data directory
DATADIR = '/content/drive/MyDrive/HAR/UCI_HAR_dataset'

In [ ]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [ ]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/HAR/UCI_HAR_dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/MyDrive/HAR/UCI_HAR_dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

In [ ]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.compat.v1.set_random_seed(42)

In [ ]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [ ]:
# Import Keras
from keras import backend as K
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [ ]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [ ]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 19s 37ms/step - loss: 1.1889 - accuracy: 0.5027 - val_loss: 0.9088 - val_accuracy: 0.6206
Epoch 2/30
460/460 [==============================] - 11s 24ms/step - loss: 0.7938 - accuracy: 0.6576 - val_loss: 0.7904 - val_accuracy: 0.6878
Epoch 3/30
460/460 [==============================] - 11s 25ms/step - loss: 0.6605 - accuracy: 0.7228 - val_loss: 0.6852 - val_accuracy: 0.7384
Epoch 4/30
460/460 [==============================] - 12s 26ms/step - loss: 0.4930 - accuracy: 0.8236 - val_loss: 0.7445 - val_accuracy: 0.7713
Epoch 5/30
460/460 [==============================] - 11s 24ms/step - loss: 0.3684 - accuracy: 0.8855 - val_loss: 0.5712 - val_accuracy: 0.8327
Epoch 6/30
460/460 [==============================] - 12s 25ms/step - loss: 0.3111 - accuracy: 0.9006 - val_loss: 0.5125 - val_accuracy: 0.8415
Epoch 7/30
460/460 [==============================] - 12s 27ms/step - loss: 0.2631 - accuracy: 0.9203 - val_loss: 0.3902 - val_accuracy:

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 1s 7ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  1      417        68        0                   3   
STANDING                 0      101       429        1                   0   
WALKING                  0        5         0      453                   1   
WALKING_DOWNSTAIRS       0        0         0        3                 414   
WALKING_UPSTAIRS         1        1         0        6                   3   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           1  
WALKING                           37  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 460  


In [ ]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 7ms/step - loss: 0.4137 - accuracy: 0.9104


In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 7ms/step - loss: 0.4137 - accuracy: 0.9104

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.4137417674064636, 0.910417377948761]


In [ ]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(Y_train)

# Bias regularizer value - we will use elasticnet
from keras import regularizers

regularizers.l1_l2(l1=0.001, l2=0.1) # the strength of l1 is set to 0.001 and l2 to 0.1


In [ ]:
# Model execution
from keras.layers import BatchNormalization 
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization (BatchN  (None, 128, 48)          192       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train,
          Y_train, 
          batch_size=n_batch,
          validation_data=(X_test, Y_test), 
          epochs= n_epochs)

Epoch 1/30
460/460 [==============================] - 31s 61ms/step - loss: 0.8558 - accuracy: 0.6842 - val_loss: 0.4024 - val_accuracy: 0.8799
Epoch 2/30
460/460 [==============================] - 27s 60ms/step - loss: 0.3732 - accuracy: 0.8814 - val_loss: 0.4981 - val_accuracy: 0.8042
Epoch 3/30
460/460 [==============================] - 27s 59ms/step - loss: 0.3064 - accuracy: 0.9008 - val_loss: 0.2548 - val_accuracy: 0.8918
Epoch 4/30
460/460 [==============================] - 28s 61ms/step - loss: 0.2206 - accuracy: 0.9285 - val_loss: 0.3191 - val_accuracy: 0.8907
Epoch 5/30
460/460 [==============================] - 27s 60ms/step - loss: 0.3269 - accuracy: 0.8966 - val_loss: 0.2696 - val_accuracy: 0.8935
Epoch 6/30
460/460 [==============================] - 28s 61ms/step - loss: 0.2296 - accuracy: 0.9234 - val_loss: 0.2965 - val_accuracy: 0.8914
Epoch 7/30
460/460 [==============================] - 28s 60ms/step - loss: 0.1803 - accuracy: 0.9378 - val_loss: 0.2297 - val_accuracy:

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 2s 16ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  4      373       112        0                   1   
STANDING                 0       66       464        0                   2   
WALKING                  0        4        12      434                  38   
WALKING_DOWNSTAIRS       0        0         0        0                 405   
WALKING_UPSTAIRS         3        1         6        7                  28   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            8  
WALKING_DOWNSTAIRS                15  
WALKING_UPSTAIRS                 426  


In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 16ms/step - loss: 0.3974 - accuracy: 0.8955

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.3974277079105377, 0.8954869508743286]


In [ ]:
# Model execution
from keras.layers import BatchNormalization
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128, 64)           18944     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_4 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_4 (Dropout)         (None, 48)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 294       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, Y_train, batch_size=n_batch, validation_data=(X_test, Y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 37s 75ms/step - loss: 0.7153 - accuracy: 0.7349 - val_loss: 0.5366 - val_accuracy: 0.8283
Epoch 2/30
460/460 [==============================] - 33s 71ms/step - loss: 0.3114 - accuracy: 0.8989 - val_loss: 0.3680 - val_accuracy: 0.8683
Epoch 3/30
460/460 [==============================] - 33s 71ms/step - loss: 0.2802 - accuracy: 0.9030 - val_loss: 0.3072 - val_accuracy: 0.9002
Epoch 4/30
460/460 [==============================] - 33s 71ms/step - loss: 0.2105 - accuracy: 0.9252 - val_loss: 0.2482 - val_accuracy: 0.9074
Epoch 5/30
460/460 [==============================] - 35s 77ms/step - loss: 0.1922 - accuracy: 0.9332 - val_loss: 0.2584 - val_accuracy: 0.9111
Epoch 6/30
460/460 [==============================] - 33s 72ms/step - loss: 0.1777 - accuracy: 0.9329 - val_loss: 0.2386 - val_accuracy: 0.9196
Epoch 7/30
460/460 [==============================] - 34s 73ms/step - loss: 0.1773 - accuracy: 0.9353 - val_loss: 0.3286 - val_accuracy:

In [ ]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

93/93 [==============================] - 3s 21ms/step
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      379       108        0                   0   
STANDING                 0       77       454        0                   1   
WALKING                  0        0         0      463                  29   
WALKING_DOWNSTAIRS       0        0         0        0                 419   
WALKING_UPSTAIRS         0        0         0        0                  21   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 450  


In [ ]:
score = model.evaluate(X_test, Y_test)

print("\n   cat_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 2s 22ms/step - loss: 0.3086 - accuracy: 0.9169

   cat_crossentropy  ||   accuracy 
  ____________________________________
[0.3086159825325012, 0.9168646335601807]
